In [1]:
import os
import torch 
from tqdm.auto import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import XLNetTokenizer, XLNetForSequenceClassification, AdamW
# 接下来，您可以使用与BERT相同的训练和验证循环逻辑，但确保所有的模型、数据和优化器都已经切换到XLNet。

In [2]:
# 加载数据
# 加载数据
new_file_path = './dataset2/OpenOffice_total_10_10.csv'
# 指定需要提取的列
columns_to_extract = ['bug_id', 'product', 'abstracts', 'description', 'component', 'severity', 'priority', 'developer',  'status','history']
# columns_to_extract = [ 'description', 'developer']
df = pd.read_csv(new_file_path, usecols=columns_to_extract, encoding='latin-1')
# 将developer列作为标签
label_dict = {label: idx for idx, label in enumerate(df['developer'].unique())}
print(f' the number of label is {len(label_dict)}')
df['label'] = df['developer'].replace(label_dict).infer_objects()
# 合并bug_id和summary作为模型的输入
df['text_input'] = df['abstracts'].astype(str) + " " + df['description'].astype(str)+ " "  + df['product'].astype(str)+ df['component'].astype(str)+ " " + df['severity'].astype(str)+ " " + df['priority'].astype(str)+ " " + df['history'].astype(str)  # 使用空格作为分隔符
X_train, X_val, y_train, y_val = train_test_split(df.index.values, df.label.values, test_size=0.15, random_state=42, stratify=df.label.values)
df['data_type'] = ['not_set']*df.shape[0]
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

 the number of label is 170


C:\Users\Administrator\AppData\Local\Temp\ipykernel_38556\3016428114.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df['developer'].replace(label_dict).infer_objects()


In [3]:
# new_file_path = 'dataprocessed_description_more_than10.csv'
# df = pd.read_csv(new_file_path, encoding='latin-1')
# df = df[['bug_id', 'summary', 'who', 'description']]
# label_dict = {label: idx for idx, label in enumerate(df['who'].unique())}
# df['label'] = df['who'].replace(label_dict)
# print(f' the number of label is {len(label_dict)}')
# # 合并bug_id和summary作为模型的输入
# df['text_input'] = df['description']  # 使用空格作为分隔符
# X_train, X_val, y_train, y_val = train_test_split(df.index.values, df.label.values, test_size=0.15, random_state=42, stratify=df.label.values)
# df['data_type'] = ['not_set']*df.shape[0]
# df.loc[X_train, 'data_type'] = 'train'
# df.loc[X_val, 'data_type'] = 'val'

In [3]:
# 使用XLNet的分词器
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

In [4]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text_input.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',  # 更新此处
    truncation=True,  # 添加此行
    max_length=512,
    return_tensors='pt'
)
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text_input.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',  # 更新此处
    truncation=True,  # 添加此行
    max_length=512,
    return_tensors='pt'
)

In [5]:
# 准备Tensor数据
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [6]:
# 定义DataLoader
batch_size = 2
train_loader = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=batch_size)
val_loader = DataLoader(dataset_val, sampler=SequentialSampler(dataset_val), batch_size=32)

In [7]:
# 初始化XLNet模型
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=len(label_dict))
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)
# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.weight', 'sequence_summary.summary.bias', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0-11): 12 x XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (sequence_summary): SequenceSummary(
    (summary): Linear(in_features=768, out_features=768, bias=True)
    (activation): Tanh()
    (first_dropout): Identity()
    (last

In [8]:
checkpoint_path = 'Xlnet_OpenOffice_total_10_10_plus.pth'
# 检查是否有可用的检查点
if os.path.isfile(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f'Resuming training from epoch {start_epoch}')
else:
    start_epoch = 0
    print('Starting training from scratch')

Starting training from scratch


In [9]:
import pymysql
from datetime import datetime

# 数据库连接信息
host = '38.147.173.234'
user = 'lijianye'
password = '660013'
db = 'training_statistics_db'
experiment_num = 13
# 模型名称，根据实际情况手动设置
model_name = 'xlnet'
# 学习率和可选特性，根据实际情况手动设置
learning_rate = 1e-5  # 示例学习率
optional_feature = 'abstract+descrition'  # 示例可选特性
dataset = new_file_path
num_epochs = 15
for epoch in range(start_epoch, num_epochs):
    model.train()
    start_time = datetime.now()
        
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}")
    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        progress_bar.set_postfix(loss=loss.item())
    torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, checkpoint_path)
    model.eval()
    correct_topk = {k: 0 for k in range(1, 11)}
    total = 0
    val_progress_bar = tqdm(val_loader, desc="Validating")
    
    for batch in val_progress_bar:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs[0]
        total += labels.size(0)
        
        # 计算top1到top10的正确率
        _, predicted_topk = torch.topk(logits, k=10, dim=1)
        labels_expanded = labels.unsqueeze(1)
        for k in range(1, 11):
            correct_topk[k] += (predicted_topk[:, :k] == labels_expanded).any(dim=1).sum().item()
                
    # 打印每个topK的准确率
    top10accuracy = []  # 初始化存储Top1到Top10准确率的数组

    for k in range(1, 11):
        accuracy = 100 * correct_topk[k] / total
        top10accuracy.append(accuracy)  # 将计算出的准确率添加到数组中
        print(f'Accuracy after epoch {epoch + 1}: Top{k}: {accuracy:.2f}%')
        print(top10accuracy)
    import pandas as pd
    import os
        # ...训练结束时间
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()/60.0
    # 定义数据字典，用于创建DataFrame
    data = {
            'epoch': [epoch],
            'start_time': [start_time],
            'end_time': [end_time],
            'duration': [duration],
            'user_id': [1],
            'model': [model_name],
            'top1_accuracy': [top10accuracy[0]],
            'top2_accuracy': [top10accuracy[1]],
            'top3_accuracy': [top10accuracy[2]],
            'top4_accuracy': [top10accuracy[3]],
            'top5_accuracy': [top10accuracy[4]],
            'top6_accuracy': [top10accuracy[5]],
            'top7_accuracy': [top10accuracy[6]],
            'top8_accuracy': [top10accuracy[7]],
            'top9_accuracy': [top10accuracy[8]],
            'top10_accuracy': [top10accuracy[9]],
            'optional_feature': [optional_feature],
            'learning_rate': [learning_rate],
            'dataset': [dataset],
            'experiment_num':[experiment_num],
    }

        # 创建DataFrame
    df = pd.DataFrame(data)

        # 检查train.csv文件是否存在来决定是否添加表头
    file_exists = os.path.isfile('modified_train_with_updated_duration.csv')

        # 如果文件存在，不写入表头，模式为追加；如果文件不存在，写入表头，模式为写入
    df.to_csv('xuezhangtrainoutcome.csv', mode='a', header=not file_exists, index=False)

    print(f'Epoch {epoch + 1} training data inserted into train.csv.')

Epoch 1:   0%|          | 0/8783 [00:00<?, ?it/s]

Validating:   0%|          | 0/97 [00:00<?, ?it/s]

Accuracy after epoch 1: Top1: 81.72%
[81.7155756207675]
Accuracy after epoch 1: Top2: 88.42%
[81.7155756207675, 88.42308932602387]
Accuracy after epoch 1: Top3: 90.84%
[81.7155756207675, 88.42308932602387, 90.8416639793615]
Accuracy after epoch 1: Top4: 91.97%
[81.7155756207675, 88.42308932602387, 90.8416639793615, 91.97033215091906]
Accuracy after epoch 1: Top5: 92.74%
[81.7155756207675, 88.42308932602387, 90.8416639793615, 91.97033215091906, 92.7442760399871]
Accuracy after epoch 1: Top6: 93.13%
[81.7155756207675, 88.42308932602387, 90.8416639793615, 91.97033215091906, 92.7442760399871, 93.13124798452112]
Accuracy after epoch 1: Top7: 93.45%
[81.7155756207675, 88.42308932602387, 90.8416639793615, 91.97033215091906, 92.7442760399871, 93.13124798452112, 93.45372460496614]
Accuracy after epoch 1: Top8: 93.81%
[81.7155756207675, 88.42308932602387, 90.8416639793615, 91.97033215091906, 92.7442760399871, 93.13124798452112, 93.45372460496614, 93.80844888745565]
Accuracy after epoch 1: Top9: 

Epoch 2:   0%|          | 0/8783 [00:00<?, ?it/s]

Validating:   0%|          | 0/97 [00:00<?, ?it/s]

Accuracy after epoch 2: Top1: 84.65%
[84.65011286681715]
Accuracy after epoch 2: Top2: 90.94%
[84.65011286681715, 90.938406965495]
Accuracy after epoch 2: Top3: 92.71%
[84.65011286681715, 90.938406965495, 92.7120283779426]
Accuracy after epoch 2: Top4: 93.78%
[84.65011286681715, 90.938406965495, 92.7120283779426, 93.77620122541116]
Accuracy after epoch 2: Top5: 94.32%
[84.65011286681715, 90.938406965495, 92.7120283779426, 93.77620122541116, 94.32441148016768]
Accuracy after epoch 2: Top6: 94.84%
[84.65011286681715, 90.938406965495, 92.7120283779426, 93.77620122541116, 94.32441148016768, 94.84037407287971]
Accuracy after epoch 2: Top7: 95.10%
[84.65011286681715, 90.938406965495, 92.7120283779426, 93.77620122541116, 94.32441148016768, 94.84037407287971, 95.09835536923573]
Accuracy after epoch 2: Top8: 95.32%
[84.65011286681715, 90.938406965495, 92.7120283779426, 93.77620122541116, 94.32441148016768, 94.84037407287971, 95.09835536923573, 95.32408900354724]
Accuracy after epoch 2: Top9: 95

Epoch 3:   0%|          | 0/8783 [00:00<?, ?it/s]

Validating:   0%|          | 0/97 [00:00<?, ?it/s]

Accuracy after epoch 3: Top1: 85.30%
[85.29506610770719]
Accuracy after epoch 3: Top2: 91.58%
[85.29506610770719, 91.58336020638504]
Accuracy after epoch 3: Top3: 93.20%
[85.29506610770719, 91.58336020638504, 93.19574330861012]
Accuracy after epoch 3: Top4: 94.23%
[85.29506610770719, 91.58336020638504, 93.19574330861012, 94.22766849403418]
Accuracy after epoch 3: Top5: 94.84%
[85.29506610770719, 91.58336020638504, 93.19574330861012, 94.22766849403418, 94.84037407287971]
Accuracy after epoch 3: Top6: 95.26%
[85.29506610770719, 91.58336020638504, 93.19574330861012, 94.22766849403418, 94.84037407287971, 95.25959367945823]
Accuracy after epoch 3: Top7: 95.55%
[85.29506610770719, 91.58336020638504, 93.19574330861012, 94.22766849403418, 94.84037407287971, 95.25959367945823, 95.54982263785875]
Accuracy after epoch 3: Top8: 95.84%
[85.29506610770719, 91.58336020638504, 93.19574330861012, 94.22766849403418, 94.84037407287971, 95.25959367945823, 95.54982263785875, 95.84005159625927]
Accuracy aft

Epoch 4:   0%|          | 0/8783 [00:00<?, ?it/s]

In [ ]:
# 训练和验证循环 增加top5 top10
num_epochs = 130  # 这里设置一个小一点的数，以便于测试，您可以根据需要调整
for epoch in range(start_epoch, num_epochs):
    model.train()
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}")
    for batch in progress_bar:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        progress_bar.set_postfix(loss=loss.item())
    torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, checkpoint_path)
    model.eval()
    correct_topk = {k: 0 for k in range(1, 11)}
    total = 0
    val_progress_bar = tqdm(val_loader, desc="Validating")
    
    for batch in val_progress_bar:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs[0]
        total += labels.size(0)
        
        # 计算top1到top10的正确率
        _, predicted_topk = torch.topk(logits, k=10, dim=1)
        labels_expanded = labels.unsqueeze(1)
        for k in range(1, 11):
            correct_topk[k] += (predicted_topk[:, :k] == labels_expanded).any(dim=1).sum().item()
    # 打印每个topK的准确率
    for k in range(1, 11):
        accuracy = 100 * correct_topk[k] / total
        print(f'Accuracy after epoch {epoch + 1}: Top{k}: {accuracy:.2f}%')

Epoch 6:   0%|          | 0/2547 [00:00<?, ?it/s]

Validating:   0%|          | 0/85 [00:00<?, ?it/s]

Accuracy after epoch 6: Top1: 54.02%
Accuracy after epoch 6: Top2: 66.67%
Accuracy after epoch 6: Top3: 73.82%
Accuracy after epoch 6: Top4: 76.75%
Accuracy after epoch 6: Top5: 78.49%
Accuracy after epoch 6: Top6: 80.20%
Accuracy after epoch 6: Top7: 81.72%
Accuracy after epoch 6: Top8: 82.91%
Accuracy after epoch 6: Top9: 84.09%
Accuracy after epoch 6: Top10: 84.72%


Epoch 7:   0%|          | 0/2547 [00:00<?, ?it/s]

Validating:   0%|          | 0/85 [00:00<?, ?it/s]

...

Epoch 11:   0%|          | 0/2547 [00:00<?, ?it/s]

Validating:   0%|          | 0/85 [00:00<?, ?it/s]

Accuracy after epoch 11: Top1: 55.02%
Accuracy after epoch 11: Top2: 66.07%
Accuracy after epoch 11: Top3: 72.30%
Accuracy after epoch 11: Top4: 75.64%
Accuracy after epoch 11: Top5: 77.72%
Accuracy after epoch 11: Top6: 79.64%
Accuracy after epoch 11: Top7: 80.94%
Accuracy after epoch 11: Top8: 82.17%
Accuracy after epoch 11: Top9: 83.24%
Accuracy after epoch 11: Top10: 83.98%


Epoch 12:   0%|          | 0/2547 [00:00<?, ?it/s]

KeyboardInterrupt: 